# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup

import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/Output/city_weather.csv")

# Display sample data
city_data_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,562245,Gadzhiyevo,RU,69.2551,33.3362,13.33,94,84,4.72
1,1546102,Port-aux-Français,TF,-49.3500,70.2167,39.34,73,97,26.66
2,2206890,Masterton,NZ,-40.9597,175.6575,52.12,95,100,5.10
3,6185377,Yellowknife,CA,62.4560,-114.3525,9.12,85,100,6.91
4,211647,Lodja,CD,-3.4833,23.4333,69.96,99,99,1.10
...,...,...,...,...,...,...,...,...,...
574,1175712,Jiwani,PK,25.0500,61.7417,76.30,28,50,12.19
575,779683,Hammerfest,NO,70.6634,23.6821,19.85,79,0,3.44
576,1079048,Ambovombe,MG,-25.1667,46.0833,70.84,80,0,14.65
577,3347353,Menongue,AO,-14.6585,17.6910,63.81,93,85,3.40


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
map_plot_1 =city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size= "Humidity",
    geo = True,
    tiles = "OSM",
    frame_width =700,
    frame_height =500,
    scale = 1,
    color = "City"
)


# Display the map

map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values

temp_df = city_data_df[(city_data_df["Temperature"] >= 75) & (city_data_df["Temperature"] <= 90)]
wind_df = temp_df[temp_df["Wind Speed"] <= 10]
weather_df = wind_df[wind_df["Clouds"] <=10]
weather_df = wind_df[wind_df["Humidity"] <=70]
weather_df.head()

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
38,245785,Abéché,TD,13.8292,20.8324,78.21,11,93,8.28
83,3354077,Opuwo,NaN,-18.0607,13.8400,75.27,62,100,4.59
99,3404889,Brejo,BR,-3.6844,-42.7503,86.27,57,41,8.41
114,2063056,Port Augusta,AU,-32.5000,137.7667,78.04,47,20,2.86
118,3407216,Arcoverde,BR,-8.4189,-37.0539,80.04,57,100,9.89


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_df.drop(columns = ["City ID", "Temperature", "Clouds", "Wind Speed"]).copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
38,Abéché,TD,13.8292,20.8324,11,
83,Opuwo,NaN,-18.0607,13.8400,62,
99,Brejo,BR,-3.6844,-42.7503,57,
114,Port Augusta,AU,-32.5000,137.7667,47,
118,Arcoverde,BR,-8.4189,-37.0539,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
# Set parameters to search for a hotel
radius = 10000
categories="accommodation.hotel"
limit= 1
params = {
    "categories":categories,
    "limit":limit,   
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{'Lon'},{'Lat'},{radius}"
    params["bias"] = f"proximity:{'Lon'},{'Lat'}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
Abéché - nearest hotel: No hotel found
Opuwo - nearest hotel: No hotel found
Brejo - nearest hotel: No hotel found
Port Augusta - nearest hotel: No hotel found
Arcoverde - nearest hotel: No hotel found
Petrolina - nearest hotel: No hotel found
Alofi - nearest hotel: No hotel found
São Gabriel da Cachoeira - nearest hotel: No hotel found
Ménaka - nearest hotel: No hotel found
Puri - nearest hotel: No hotel found
Katima Mulilo - nearest hotel: No hotel found
Mineiros - nearest hotel: No hotel found
Cabo San Lucas - nearest hotel: No hotel found
Rābigh - nearest hotel: No hotel found
Acajutla - nearest hotel: No hotel found
Lázaro Cárdenas - nearest hotel: No hotel found
Bitkine - nearest hotel: No hotel found
Nicoya - nearest hotel: No hotel found
Coari - nearest hotel: No hotel found
Baiti - nearest hotel: No hotel found
Koné - nearest hotel: No hotel found
Sur - nearest hotel: No hotel found
Tefé - nearest hotel: No hotel found
La Mira - nearest hotel: No hotel fo

,City,Country,Lat,Lng,Humidity,Hotel Name
38,Abéché,TD,13.8292,20.8324,11,No hotel found
83,Opuwo,NaN,-18.0607,13.8400,62,No hotel found
99,Brejo,BR,-3.6844,-42.7503,57,No hotel found
114,Port Augusta,AU,-32.5000,137.7667,47,No hotel found
118,Arcoverde,BR,-8.4189,-37.0539,57,No hotel found
146,Petrolina,BR,-9.3986,-40.5008,58,No hotel found
171,Alofi,NU,-19.0595,-169.9187,70,No hotel found
173,São Gabriel da Cachoeira,BR,-0.1303,-67.0892,62,No hotel found
180,Ménaka,ML,15.9182,2.4022,14,No hotel found
182,Puri,IN,19.8000,85.8500,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width =600,
    frame_height =600,
    scale = 1,
    color = "City",
    hover_cols = "all"
)
# Display the map

map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)